## Ejercicio: Analizar texto

Utilizar un texto de proyecto Gutenberg en castellano (http://www.gutenberg.org/browse/languages/es)

Contar palabras y ordenar por frecuencia
- Limpiar preludio y licencia de Project Gutenberg
- Omitir “palabras vacías” (stop words) y símbolos

Encontrar personajes

Hacer un análisis extra a gusto

In [1]:
from collections import Counter
# función auxiliar
#https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/
def leer_texto(texto):
    """Funcion auxiliar para leer un archivo de texto"""
    with open(texto, 'r') as text:
        return text.read()
    
def inStr_idx(textin,strLst):
    s_id_ini=[]
    s_id_end=[]
    for s in strLst:
        s_len=len(s)
        if str.find(textin,s)!=-1:
            s_id_ini.append(str.find(textin,s))
            s_id_end.append(str.find(textin,s)+s_len)
        else:
            s_id_ini.append(-1)
            s_id_end.append(-1)
    return s_id_ini,s_id_end

#https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/
def encontrar_personajes(doc):
    """
    Devuelve una lista de los personajes de un `doc` con su cantidad de
    ocurrencias
    
    :param doc: NLP documento parseado por Spacy
    :return: Lista de Tuplas con la forma
        [('winston', 686), ("o'brien", 135), ('julia', 85),]
    """
    personajes = Counter()
    for ent in doc.ents:
        if ent.label_ == 'PER':
            personajes[ent.lemma_] += 1
            
    return personajes.most_common()

#https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/
def obtener_adj_pers(doc, personaje):
    """
    Encontrar todos los adjetivos relacionados a un personaje en un `doc`
    
    :param doc: NLP documento parseado por Spacy
    :param personaje: un objeto String 
    :return: lista de adjetivos relacionados a un `personaje`
    """
    
    adjetivos = []
    for ent in doc.ents:
        if ent.lemma_ == personaje:
            for token in ent.subtree:
                if token.pos_ == 'ADJ':
                    adjetivos.append(token.lemma_)
    
    for ent in doc.ents:
        if ent.lemma_ == personaje:
            if ent.root.dep_ == 'nsubj':
                for child in ent.root.head.children:
                    if child.dep_ == 'acomp':
                        adjetivos.append(child.lemma_)
    
    return adjetivos

*Limpiar preludio y licencia de Project Gutenberg*

In [2]:
# Cargamos el texto
texto_raw = leer_texto('pg43033.txt')
# Vemos de limpiar la parte de proyecto Gutenberg. Si abrimos el documento observamos un par de frases en las cuales parecen marcar el limite
# ['Nota del transcriptor: La ortografía del original fue conservada.','End of Project Gutenberg']
# Entonces buscamos los indices final de la primera e inicial de la segunda y nos quedamos con la seccion del medio
ini_idx,end_idx=inStr_idx(texto_raw,['Nota del transcriptor: La ortografía del original fue conservada.','End of Project Gutenberg'])
print(ini_idx)
print(end_idx)
texto_ini=texto_raw[end_idx[0]:ini_idx[1]]
# Y si los buscamos nuevamente no deberiamos encontrarlos (-1 en inStr_idx)
ini_idx,end_idx=inStr_idx(texto_ini,['Nota del transcriptor: La ortografía del original fue conservada.','End of Project Gutenberg'])
print(ini_idx)
print(end_idx)


[762, 397086]
[827, 397110]
[-1, -1]
[-1, -1]


Para la limpieza en el sentido automatico se podria buscar marcadores estandar para todos los libros y utlizarlos para remover las secciones correspondientes. En el presente caso "End of Project Gutenberg" parece ser un buen marcardor final, pero no me queda claro como remover la seccion inicial en forma automatica.

*Cargamos los modulos e instanciamos el modelo en español*

In [3]:
import textacy
# Cargando el modelo en español de spacy
nlp = textacy.load_spacy('es_core_news_md')

*Limpiamos un poco mas el texto. Seguimos *
- https://textacy.readthedocs.io/en/stable/api_reference.html#module-textacy.preprocess

In [4]:
# Normalizamos espacios en blanco
texto_nwp=textacy.preprocess.normalize_whitespace(texto_ini)
#Y Seguimos
texto_prc=textacy.preprocess.preprocess_text(texto_nwp, fix_unicode=True, lowercase=False, transliterate=False, no_urls=False, no_emails=False, no_phone_numbers=True, no_numbers=False, no_currency_symbols=True, no_punct=True, no_contractions=False, no_accents=True)

In [5]:
# Ahora lo incorporamos en un doc de textacy
doc=textacy.Doc(texto_prc,lang=nlp)

In [6]:
bag_terms=doc.to_bag_of_terms(ngrams=(1), named_entities=True, normalize=u'lemma', lemmatize=None, lowercase=None, weighting=u'freq', as_strings=True,filter_stops=True,filter_punct=True)

*Las listas ordenadas por frecuencia serian:*

In [7]:
# De : https://www.pythoncentral.io/how-to-sort-python-dictionaries-by-key-or-value/
bag_terms_list_sorted_min_max=sorted(bag_terms, key=bag_terms.__getitem__,reverse=False)
bag_terms_list_sorted_max_min=sorted(bag_terms, key=bag_terms.__getitem__,reverse=True)

*Buscamos los personajes*

In [8]:
#Usamos extract.named_entities
ents=textacy.extract.named_entities(doc)
ents_per=[]
ents_lemma_set=[]
for x in ents:
    ents_lemma_set.append(x.label_)
    #En https://textacy.readthedocs.io/en/stable/api_reference.html#module-textacy.extract menciona PERSON, pero al ver los posibles lemmas solo PER existe como opcion...
    if x.label_=='PER':
        ents_per.append(x.lemma_)
ents_lemma_set=list(set(ents_lemma_set))
ents_per=list(set(ents_per))
print(ents_lemma_set)
print(len(ents_per))
ents_per[:10]

['PER', 'MISC', 'LOC', 'ORG']
838


['',
 'Bayardo',
 'Comprendio',
 'Maximo',
 'Muley Edris',
 'Pero Vidal',
 'Quizas',
 'Matansa',
 'Manuel Asi asi Te',
 'Don Alonso']

*Usamos alternativamente la funcion del tutorial. No me queda claro porque interpretaria ' ' como PER*

In [9]:
ents_per_func=encontrar_personajes(doc.spacy_doc)
len(ents_per_func)

844

*Vemos que tenemos alguna diferencia en el metodo. Veamos de donde sale...*

In [10]:
entlist=[x[0] for x in ents_per_func]
diff_list_a=[];diff_list_b=[]
for x in entlist:
    if x in ents_per:
        pass
    else:
        diff_list_a.append(x)
for x in ents_per:
    if x in entlist:
        pass
    else:
        diff_list_b.append(x)

In [11]:
diff_list_a

['El Bizco',
 'El',
 'La',
 'El Garro',
 'Un',
 'Las \n cuco',
 'La baronesa de Aynant Paquita Figueroa',
 'Los',
 'Aquel Bayardo']

In [12]:
diff_list_b

['Bayardo', 'baronesa de Aynant Paquita Figueroa', 'cuco']

In [13]:
# Y la lista de Stop Words
stop_words=list(nlp.Defaults.stop_words)
#Las primeras 20
stop_words[0:20]

['actualmente',
 'paìs',
 'dan',
 'ciertas',
 'decir',
 'haya',
 'eramos',
 'estan',
 'soy',
 'realizó',
 'al',
 'vuestros',
 'cuándo',
 'somos',
 'he',
 'mismas',
 'era',
 'cuántas',
 'misma',
 'consideró']

In [14]:
# Veamos que efectivamente "El,La,Un,Los,La,Aquel" son Stops Words.
str_test=list(set([x.split()[0] for x in diff_list_a]))
str_test
str_in_stop_words=[]
for x in str_test:
    if x.lower() in stop_words:
        str_in_stop_words.append((x,'OK'))
    else:
        str_in_stop_words.append((x,'FAIL'))
str_in_stop_words

[('El', 'OK'),
 ('Aquel', 'OK'),
 ('Las', 'OK'),
 ('La', 'OK'),
 ('Los', 'OK'),
 ('Un', 'OK')]

*Vemos que en el caso b no figuran las stop words. La diferencia parece estar por ese sector. Igualmente las palabras "Garro" y "Bizco" no se porque no figuran en ambas listas.*

*En terminos de extra, seguimos al tutorial y recolectamos aquellas cosas que caracterizan a una palabra. En este caso: "Manuel"*

In [15]:
print(obtener_adj_pers(doc.spacy_doc, "Manuel"))

['nuevo', 'duro', 'creyo', 'manana', 'altivar', 'noble', 'blanco', 'soso', 'nina', 'solo', 'propiciar', 'despues', 'comun', 'despues', 'enterar', 'albanil', 'alegrar', 'cuartar', 'repatriar', 'tias', 'escandaloso', 'solo', 'advertir', 'acompano', 'nuevo', 'insoportable', 'creyo', 'recorrio', 'irritar', 'indeciso']


In [16]:
#Otra cosa que parece resaltar es que se selecciona personajes por comenzar en mayuscula
# Entonces si tomamos todo en minusculas no deberiamos ver escencialmente ningun PER
texto_prc=textacy.preprocess.preprocess_text(texto_nwp, fix_unicode=True, lowercase=True, transliterate=False, no_urls=False, no_emails=False, no_phone_numbers=True, no_numbers=False, no_currency_symbols=True, no_punct=True, no_contractions=False, no_accents=True)
# Ahora lo incorporamos en un doc de textacy
doc=textacy.Doc(texto_prc,lang=nlp)

In [17]:
ents=textacy.extract.named_entities(doc)
ents_per=[]
ents_lemma_set=[]
for x in ents:
    ents_lemma_set.append(x.label_)
    #En https://textacy.readthedocs.io/en/stable/api_reference.html#module-textacy.extract menciona PERSON, pero al ver los posibles lemmas solo PER existe como opcion...
    if x.label_=='PER':
        ents_per.append(x.lemma_)
ents_lemma_set=list(set(ents_lemma_set))
ents_per=list(set(ents_per))
print(ents_lemma_set)
print(len(ents_per))
ents_per[:10]

['PER', 'LOC', 'MISC', 'ORG']
1


['']

Que es justamente lo que comentabamos en relacion a la seleccion de personajes por comenzar en mayuscula.